# Funções de Perda

O módulo ```nn``` e suas 1001 utilidades, também fornece as implementações das principais funções de perda. Então vamos primeiro importar o ```torch``` e o módulo ```nn``` <br>

In [1]:
import torch
from torch import nn

Antes de tudo, vamos conferir qual dispositivo de hardware está disponível para uso.

In [2]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


Vamos trabalhar com o dataset de classificação de vinhos.

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html

Classes: 3
Samples per class: [59,71,48]
Samples total: 178
Dimensionality: 13
Features: real, positive

In [3]:
from sklearn import datasets

wine = datasets.load_wine()
data = wine.data
target = wine.target

print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)

(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']



Vamos instanciar um MLP com uma camada escondida e uma camada de saída. <br>

In [4]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden  = nn.Linear(input_size, hidden_size)
    self.relu    = nn.ReLU()
    self.out     = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax()

  def forward(self, X):
    
    feature = self.relu(self.hidden(X))
    output  = self.softmax(self.out(feature))

    return output

input_size  = data.shape[1]
hidden_size = 32
out_size    = len(wine.target_names)

net = WineClassifier(input_size, hidden_size, out_size).to(device) #cast na GPU 

In [5]:
print(net)

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)


## Classificação

O primeiro passo é instanciar a função de perda de sua escolha. Trata-se de um problema de classificação com 3 classes, nesse caso a Cross Entropy é a função recomendada, que no PyTorch recebe o nome de *CrossEntropyLoss*: https://pytorch.org/docs/stable/nn.html#crossentropyloss 

**Assim como a rede, as entradas e os rótulos, a função de perda também deve ser carregada na GPU**


In [6]:
criterion = nn.CrossEntropyLoss().to(device) # cast na GPU

Antes de aplicar a função de perda, vamos fazer o cast dos dados para tensores e extrair as predições ```y'``` da rede.

In [7]:
Xtns = torch.from_numpy(data).float()
Ytns = torch.from_numpy(target)

# Cast na GPU
Xtns = Xtns.to(device)
Ytns = Ytns.to(device)

print(Xtns.dtype, Ytns.dtype)

torch.float32 torch.int64


In [8]:
pred = net(Xtns)

<ipython-input-4-704cff75ca11>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output  = self.softmax(self.out(feature))


Confira as dimensões de ```y``` e ```y'```. Enquanto as predições estão em termos de probabilidades, os rótulos de classificação devem são valores inteiros referentes aos índices das classes.

In [9]:
print(pred.shape, Ytns.shape)

print(pred[0].data, Ytns[0].data)

torch.Size([178, 3]) torch.Size([178])
tensor([9.1352e-07, 7.7333e-01, 2.2667e-01], device='cuda:0') tensor(0, device='cuda:0')


As funções de perda implementadas no PyTorch esperam o seguinte padrão de chamada:

```python
loss = criterion(prediction, target)
```

Vale lembrar que cada função de perda possui especificidades quanto às dimensões dos seus parâmetros. Para a Cross Entropy:
* prediction: ```(N, C)```
* target: ```(N,)```

In [10]:
loss = criterion(pred, Ytns)
print(loss)

tensor(1.2659, device='cuda:0', grad_fn=<NllLossBackward0>)


## Regressão


Vamos trabalhar com o dataset de Diabetes, cujo objetivo é prever a progressão da diabetes em um paciente.

https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset

In [11]:
from sklearn import datasets

diabetes = datasets.load_diabetes()
data = diabetes.data
target = diabetes.target

print(data.shape, target.shape)

print(data[14])
print(target[14])

(442, 10) (442,)
[ 4.53409833e-02 -4.46416365e-02 -2.56065715e-02 -1.25561242e-02
  1.76943802e-02 -6.12835791e-05  8.17748397e-02 -3.94933829e-02
 -3.19876395e-02 -7.56356220e-02]
118.0


Implementando o MLP

In [12]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden  = nn.Linear(input_size, hidden_size)
    self.relu    = nn.ReLU()
    self.out     = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, X):
    
    feature = self.relu(self.hidden(X))
    output  = self.softmax(self.out(feature))

    return output

input_size  = data.shape[1]
hidden_size = 32
out_size    = 1  # Progressão da diabetes

net = WineClassifier(input_size, hidden_size, out_size).to(device) #cast na GPU 

Para solucionar problemas de regressão, as funções de perda correspondentes esperam que ambos o rótulo e a predição tenham **a mesma dimensionalidade**. Não se trata mais de um problema categórico.

Portanto, vamos simular um problema de regressão e aplicar a *MSELoss*<br>
Documentação: https://pytorch.org/docs/stable/nn.html#mseloss

In [13]:
criterion = nn.MSELoss().to(device)

# Cast na GPU
Xtns = torch.from_numpy(data).float().to(device)
Ytns = torch.from_numpy(target).float().to(device)

print(Xtns.shape, Ytns.shape)

torch.Size([442, 10]) torch.Size([442])


In [14]:
pred = net(Xtns)

loss = criterion(pred.squeeze(), Ytns)
print(loss.data)

tensor(28771.2168, device='cuda:0')


In [15]:
criterion = nn.L1Loss().to(device)

pred = net(Xtns)

loss = criterion(pred.squeeze(), Ytns)
print(loss.data)

tensor(151.1335, device='cuda:0')


## Documentação
Veja a documentação para consultar a lista de todas as funções de perda implementadas no PyTorch: <br>
https://pytorch.org/docs/stable/nn.html#loss-functions